In [1]:
import tensorflow as tf
import scipy, numpy

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

2023-11-23 19:02:44.374309: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-23 19:02:44.412078: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-23 19:02:44.412733: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-23 19:02:45.206564: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
# Configurações para o treinamento
train_data_dir = './data'
batch_size = 32 # 32 imagens por lote
num_epochs = 100 # 100 épocas de treinamento
image_size = (150, 150) # 150x150 pixels
num_classes = 2 # 2 classes: 'Papai Noel' e 'Não Papai Noel'

In [3]:
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    validation_split = 0.25, # 25% para validação
    shear_range = 0.2, # inclinação
    zoom_range = 0.2, # zoom
    horizontal_flip = True # espelhamento horizontal
)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'training'
)

Found 462 images belonging to 2 classes.


In [5]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'validation'
)

Found 152 images belonging to 2 classes.


In [6]:
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(image_size[0], image_size[1], 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [7]:
# compilar o modelo
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy', # mais próximo de 0 melhor, 0.002 por exemplo
    metrics = ['accuracy']
)

In [11]:
# treinamento
model.fit(
    train_generator,
    epochs = num_epochs,
    validation_data = validation_generator
)

print('Treinamento concluído!')

Epoch 1/100
15/15 [==============================] - 23s 2s/step - loss: 0.5881 - accuracy: 0.8009 - val_loss: 0.4972 - val_accuracy: 0.8618
Epoch 2/100
15/15 [==============================] - 27s 2s/step - loss: 0.5481 - accuracy: 0.8528 - val_loss: 0.5936 - val_accuracy: 0.8026
Epoch 3/100
15/15 [==============================] - 26s 2s/step - loss: 0.5558 - accuracy: 0.7965 - val_loss: 0.5187 - val_accuracy: 0.8355
Epoch 4/100
15/15 [==============================] - 26s 2s/step - loss: 0.4916 - accuracy: 0.8745 - val_loss: 0.4476 - val_accuracy: 0.8684
Epoch 5/100
15/15 [==============================] - 26s 2s/step - loss: 0.4351 - accuracy: 0.8788 - val_loss: 0.3746 - val_accuracy: 0.8684
Epoch 6/100
15/15 [==============================] - 26s 2s/step - loss: 0.3960 - accuracy: 0.8680 - val_loss: 0.3706 - val_accuracy: 0.8487
Epoch 7/100
15/15 [==============================] - 26s 2s/step - loss: 0.3414 - accuracy: 0.8723 - val_loss: 0.2873 - val_accuracy: 0.8947
Epoch 8/100
1

In [12]:
# salvar o modelo
model.save('./models/rcastilhos_is-noel.keras')